# Genetic algorithm tests for the Courier Problem

In [22]:
import sys

sys.path.append("../src")

In [23]:
from itertools import chain, combinations


def get_all_subsets(lst):
    return list(chain.from_iterable(combinations(lst, r) for r in range(len(lst) + 1)))

In [24]:
from ga.mutations import (
    CouriersMutation,
    Mutation,
    PackagesMutation,
    RouteMutation,
    UnusedVehiclesMutation,
    UsedVehiclesMutation,
)

mutations: list[Mutation] = [
    UsedVehiclesMutation,
    UnusedVehiclesMutation,
    CouriersMutation,
    PackagesMutation,
    RouteMutation,
]

muatation_suite = get_all_subsets(mutations)

In [25]:
import glob
import os

from model import Problem
from problem_initializer import ProblemInitializer

data_dir = "data"
json_files = glob.glob(os.path.join(data_dir, "*.json"))

data: list[tuple[str, Problem]] = []
for json_file in json_files:
    initializer = ProblemInitializer()
    initializer.generate_from_json(json_file)
    testcase_name = os.path.basename(json_file).replace(".json", "")
    problem = initializer.get_problem()
    data.append((testcase_name, problem))

In [ ]:
GA_RUN_PATIENCE = 10
GA_INITIAL_POPULATION_SIZE = 50
GA_MAX_RUN_ITERATIONS = 200
GA_RUN_REPEAT = 10

In [ ]:
import time

import numpy as np
from tqdm.notebook import tqdm

from ga import GA
from model import Solution


def run_genetic_algorithm(
    problem: Problem,
    mutations: list[Mutation],
    population: list[Solution],
    repeats: int = GA_RUN_REPEAT,
) -> dict[str, float]:
    best_costs = []
    iterations_list = []
    times = []
    cost_func_evals = []

    outer_bar = tqdm(range(repeats), desc="Repeats", leave=False, position=2)
    for _ in outer_bar:
        best_solution_cost = np.inf
        patience = GA_RUN_PATIENCE
        num_iterations = 0
        ga = GA(problem=problem, mutations=mutations, initial_population=population)

        start_time = time.perf_counter()

        for solution in ga.run(max_iter=GA_MAX_RUN_ITERATIONS, verbose=False):
            current_cost = ga.get_cost(solution)

            if current_cost < best_solution_cost:
                best_solution_cost = current_cost
                patience = GA_RUN_PATIENCE
            else:
                patience -= 1

            if patience <= 0:
                break

            num_iterations += 1

        elapsed_time = time.perf_counter() - start_time

        best_costs.append(best_solution_cost)
        iterations_list.append(num_iterations)
        times.append(elapsed_time)
        cost_func_evals.append(ga._cost_function_runs)

    stats = {
        "cost_mean": np.mean(best_costs),
        "cost_std": np.std(best_costs),
        "best_max": max(best_costs),
        "cost_min": min(best_costs),
        "cost_median": np.median(best_costs),
        "iterations_mean": np.mean(iterations_list),
        "iterations_std": np.std(iterations_list),
        "iterations_min": min(iterations_list),
        "iterations_max": max(iterations_list),
        "iterations_median": np.median(iterations_list),
        "time_mean": np.mean(times),
        "time_std": np.std(times),
        "time_min": min(times),
        "time_max": max(times),
        "time_median": np.median(times),
        "cost_func_evals_min": min(cost_func_evals),
        "cost_func_evals_max": max(cost_func_evals),
        "cost_func_evals_mean": np.mean(cost_func_evals),
        "cost_func_evals_std": np.std(cost_func_evals),
        "cost_func_evals_median": np.median(cost_func_evals),
    }

    return stats

: 

In [ ]:
import pandas as pd

from generator import Generator

results_list = []

testcase_bar = tqdm(data, desc="Testcases", position=0, leave=True)

for testcase_name, problem in testcase_bar:
    testcase_bar.set_description(f"Testcase: {testcase_name}")

    generator = Generator(problem=problem)
    population = generator.generate_many_feasible(
        num_to_find=GA_INITIAL_POPULATION_SIZE, max_attempts=1000, verbose=False
    )

    mutation_bar = tqdm(
        muatation_suite, desc="Mutation Suites", position=1, leave=False
    )

    for mutation_suite in mutation_bar:
        mutation_bar.set_description(
            f"Mutations: {[m.__name__ for m in mutation_suite]}"
        )

        results = run_genetic_algorithm(
            problem=problem,
            mutations=mutation_suite,
            population=population,
        )

        row = {
            "testcase": testcase_name,
            "mutation_suite": [m.__name__ for m in mutation_suite],
            **results,
        }
        results_list.append(row)

    results = pd.DataFrame(results_list)
    results.to_parquet(f"results/{testcase_name}_results.parquet", index=False)

Testcases:   0%|          | 0/2 [00:00<?, ?it/s]

Mutation Suites:   0%|          | 0/32 [00:00<?, ?it/s]

Repeats:   0%|          | 0/10 [00:00<?, ?it/s]

Repeats:   0%|          | 0/10 [00:00<?, ?it/s]

Repeats:   0%|          | 0/10 [00:00<?, ?it/s]

...

Repeats:   0%|          | 0/10 [00:00<?, ?it/s]

Repeats:   0%|          | 0/10 [00:00<?, ?it/s]

Repeats:   0%|          | 0/10 [00:00<?, ?it/s]

Repeats:   0%|          | 0/10 [00:00<?, ?it/s]

Repeats:   0%|          | 0/10 [00:00<?, ?it/s]